In [1]:
#pip install sodapy

In [1]:
#for reading another file.  I have API credentials in another file.
from configparser import ConfigParser

import requests # library to handle requests
import json # library to handle JSON files
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#this is a library for working with data provided by Socrata
from sodapy import Socrata

#pandas for dataframes
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



#data science math and visualiztion libraries
import numpy as np
import random # library for random number generation
import sklearn
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # plotting library

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML

print('Libraries imported.')

Libraries imported.


Retrieving my credentials and setting up my query.

In [2]:
parser = ConfigParser()
_ = parser.read('seattle_key')
APP_TOKEN = parser.get('seattle_key', 'token')
#print(APP_TOKEN)

requests_url = "https://data.seattle.gov/resource/33kz-ixgy.json?$limit=1000&$where=original_time_queued%3E%3D'12/01/2020%25'"
print(requests_url)

https://data.seattle.gov/resource/33kz-ixgy.json?$limit=1000&$where=original_time_queued%3E%3D'12/01/2020%25'


This is an API request using the requests library.

In [38]:
#headers={'X-App-Token': APP_TOKEN}
#results = requests.get(requests_url, headers=headers).json()
##print(results)
        
#df = pd.DataFrame(results)
#df.shape

(1000, 11)

I found the Socrata library and wanted to use it to retrieve the data instead of the requests library.  
It's very user friendly.

In [3]:
client = Socrata("data.seattle.gov", APP_TOKEN)

# Returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("33kz-ixgy", limit=1000, where="original_time_queued >= '12/01/2020%'")

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

df.shape
#df.head()

(1000, 11)

In [4]:
df.head()

,arrived_time,beat,cad_event_number,call_type,event_clearance_description,final_call_type,initial_call_type,original_time_queued,precinct,priority,sector
0,Dec 01 2020 00:36:11:000AM,U3,2020000332450,"TELEPHONE OTHER, NOT 911",REPORT WRITTEN (NO ARREST),--THEFT - ALL OTHER,AUTO THEFT - VEH THEFT OR THEFT & RECOVERY,12/01/2020 00:00:17 AM,NORTH,3,UNION
1,Dec 01 2020 00:09:40:000AM,R3,2020000332451,"TELEPHONE OTHER, NOT 911",REPORT WRITTEN (NO ARREST),--CRISIS COMPLAINT - GENERAL,SFD - ASSIST ON FIRE OR MEDIC RESPONSE,12/01/2020 00:02:29 AM,SOUTH,1,ROBERT
2,Dec 01 2020 01:52:22:000AM,L1,2020000332453,911,NO POLICE ACTION POSSIBLE OR NECESSARY,--DISTURBANCE - OTHER,"DISTURBANCE, MISCELLANEOUS/OTHER",12/01/2020 00:08:40 AM,NORTH,3,LINCOLN
3,Dec 01 2020 00:22:32:000AM,C1,2020000332454,ALARM CALL (NOT POLICE ALARM),FALSE COMPLAINT/UNFOUNDED,"--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)","ALARM - COMM, SILENT/AUD BURG (INCL BANKS)",12/01/2020 00:09:23 AM,EAST,3,CHARLIE
4,Dec 01 2020 00:10:53:000AM,E3,2020000332455,911,ASSISTANCE RENDERED,--INTOX & LIQ VIOLS - INTOXICATED PERSON,DETOX - REQUEST FOR,12/01/2020 00:09:44 AM,EAST,3,EDWARD


In [5]:
df.rename(columns={'cad_event_number':'cad','event_clearance_description':'clearance','final_call_type':'final_type','initial_call_type':'initial_type','original_time_queued':'queued','arrived_time':'arrived'}, inplace=True)

In [6]:
df2 = df[df.columns[[2,7,0,3,9,6,5,4,8,10,1]]]
df2.head()

,cad,queued,arrived,call_type,priority,initial_type,final_type,clearance,precinct,sector,beat
0,2020000332450,12/01/2020 00:00:17 AM,Dec 01 2020 00:36:11:000AM,"TELEPHONE OTHER, NOT 911",3,AUTO THEFT - VEH THEFT OR THEFT & RECOVERY,--THEFT - ALL OTHER,REPORT WRITTEN (NO ARREST),NORTH,UNION,U3
1,2020000332451,12/01/2020 00:02:29 AM,Dec 01 2020 00:09:40:000AM,"TELEPHONE OTHER, NOT 911",1,SFD - ASSIST ON FIRE OR MEDIC RESPONSE,--CRISIS COMPLAINT - GENERAL,REPORT WRITTEN (NO ARREST),SOUTH,ROBERT,R3
2,2020000332453,12/01/2020 00:08:40 AM,Dec 01 2020 01:52:22:000AM,911,3,"DISTURBANCE, MISCELLANEOUS/OTHER",--DISTURBANCE - OTHER,NO POLICE ACTION POSSIBLE OR NECESSARY,NORTH,LINCOLN,L1
3,2020000332454,12/01/2020 00:09:23 AM,Dec 01 2020 00:22:32:000AM,ALARM CALL (NOT POLICE ALARM),3,"ALARM - COMM, SILENT/AUD BURG (INCL BANKS)","--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)",FALSE COMPLAINT/UNFOUNDED,EAST,CHARLIE,C1
4,2020000332455,12/01/2020 00:09:44 AM,Dec 01 2020 00:10:53:000AM,911,3,DETOX - REQUEST FOR,--INTOX & LIQ VIOLS - INTOXICATED PERSON,ASSISTANCE RENDERED,EAST,EDWARD,E3


In [7]:
df2.set_index('cad', drop=True, inplace=True, verify_integrity=True)
df2.shape

(1000, 10)

In [9]:
df2.head(5)

,queued,arrived,call_type,priority,initial_type,final_type,clearance,precinct,sector,beat
cad,,,,,,,,,,
2020000332450,12/01/2020 00:00:17 AM,Dec 01 2020 00:36:11:000AM,"TELEPHONE OTHER, NOT 911",3,AUTO THEFT - VEH THEFT OR THEFT & RECOVERY,--THEFT - ALL OTHER,REPORT WRITTEN (NO ARREST),NORTH,UNION,U3
2020000332451,12/01/2020 00:02:29 AM,Dec 01 2020 00:09:40:000AM,"TELEPHONE OTHER, NOT 911",1,SFD - ASSIST ON FIRE OR MEDIC RESPONSE,--CRISIS COMPLAINT - GENERAL,REPORT WRITTEN (NO ARREST),SOUTH,ROBERT,R3
2020000332453,12/01/2020 00:08:40 AM,Dec 01 2020 01:52:22:000AM,911,3,"DISTURBANCE, MISCELLANEOUS/OTHER",--DISTURBANCE - OTHER,NO POLICE ACTION POSSIBLE OR NECESSARY,NORTH,LINCOLN,L1
2020000332454,12/01/2020 00:09:23 AM,Dec 01 2020 00:22:32:000AM,ALARM CALL (NOT POLICE ALARM),3,"ALARM - COMM, SILENT/AUD BURG (INCL BANKS)","--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)",FALSE COMPLAINT/UNFOUNDED,EAST,CHARLIE,C1
2020000332455,12/01/2020 00:09:44 AM,Dec 01 2020 00:10:53:000AM,911,3,DETOX - REQUEST FOR,--INTOX & LIQ VIOLS - INTOXICATED PERSON,ASSISTANCE RENDERED,EAST,EDWARD,E3


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
arrived         1000 non-null object
beat            1000 non-null object
cad             1000 non-null object
call_type       1000 non-null object
clearance       1000 non-null object
final_type      1000 non-null object
initial_type    1000 non-null object
queued          1000 non-null object
precinct        1000 non-null object
priority        1000 non-null object
sector          975 non-null object
dtypes: object(11)
memory usage: 86.0+ KB


In [16]:
df2['arrived'] = pd.to_datetime(df2['arrived'], format="%b %d %Y %H:%M:%S:000%p")

C:\Users\DaveFunk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df2['queued'] = pd.to_datetime(df2['queued'], format="%m/%d/%Y %H:%M:%S %p")

C:\Users\DaveFunk\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 2020000332450 to 2009000416777
Data columns (total 10 columns):
queued          1000 non-null datetime64[ns]
arrived         1000 non-null datetime64[ns]
call_type       1000 non-null object
priority        1000 non-null object
initial_type    1000 non-null object
final_type      1000 non-null object
clearance       1000 non-null object
precinct        1000 non-null object
sector          975 non-null object
beat            1000 non-null object
dtypes: datetime64[ns](2), object(8)
memory usage: 125.9+ KB


In [ ]:
df2['response_time']=df2['arrived']-df2['queued']

In [ ]:
df2 = df2[df2.columns[[0,1,10,2,3,4,5,6,7,8,9]]]

In [36]:
df2['response_time'] = df2['response_time']/ np.timedelta64(1,'m')
df2.head(5)

,queued,arrived,response_time,call_type,priority,initial_type,final_type,clearance,precinct,sector,beat
cad,,,,,,,,,,,
2020000332450,2020-12-01 00:00:17,2020-12-01 00:36:11,35.900000,"TELEPHONE OTHER, NOT 911",3,AUTO THEFT - VEH THEFT OR THEFT & RECOVERY,--THEFT - ALL OTHER,REPORT WRITTEN (NO ARREST),NORTH,UNION,U3
2020000332451,2020-12-01 00:02:29,2020-12-01 00:09:40,7.183333,"TELEPHONE OTHER, NOT 911",1,SFD - ASSIST ON FIRE OR MEDIC RESPONSE,--CRISIS COMPLAINT - GENERAL,REPORT WRITTEN (NO ARREST),SOUTH,ROBERT,R3
2020000332453,2020-12-01 00:08:40,2020-12-01 01:52:22,103.700000,911,3,"DISTURBANCE, MISCELLANEOUS/OTHER",--DISTURBANCE - OTHER,NO POLICE ACTION POSSIBLE OR NECESSARY,NORTH,LINCOLN,L1
2020000332454,2020-12-01 00:09:23,2020-12-01 00:22:32,13.150000,ALARM CALL (NOT POLICE ALARM),3,"ALARM - COMM, SILENT/AUD BURG (INCL BANKS)","--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)",FALSE COMPLAINT/UNFOUNDED,EAST,CHARLIE,C1
2020000332455,2020-12-01 00:09:44,2020-12-01 00:10:53,1.150000,911,3,DETOX - REQUEST FOR,--INTOX & LIQ VIOLS - INTOXICATED PERSON,ASSISTANCE RENDERED,EAST,EDWARD,E3


In [38]:
df2.to_csv('C:/Users/DaveFunk/Desktop/call_data/call_data.csv')